In [49]:
import glob

from bs4 import BeautifulSoup as bs
import bs4

In [52]:
for fn in sorted(glob.glob('../data/annotations/*.xml')):
    with open(fn) as f:
        annotation = bs(f.read(), "lxml").find("text")
    for ad in annotation.find('div'): # each div only has one p?
        for node in ad:
            if isinstance(node, bs4.element.NavigableString):
                print('text:', node)
            else:
                print('tag:', node)

tag: 

tag: <span ana="#begin" xml:id="recogito-b2bdce14-7852-4fd1-b44d-44f56c1a118c">By Mr CO</span>
text: 
          On Monday next, the 16th inftant, and the following Doy. HE 
tag: <span ana="#Descriptor #household" xml:id="recogito-dbdf9ade-d88f-4e7e-97b2-8a8ecbb930c7">HOUSHOLD</span>
text: 

tag: <span ana="#furniture" xml:id="recogito-86b9b9f0-c5ac-46ea-bf8c-badca14ef8e5">FURNITURE</span>
text: , 
tag: <span ana="#picture" xml:id="recogito-dadeaac3-b976-4c72-963d-97afdd2eda26">PICTURES</span>
text: , and 
tag: <span ana="#Descriptor #other" xml:id="recogito-0d1399df-d925-437c-8d3c-06d5acfaa781">other</span>
text: 

tag: <span ana="#effect" xml:id="recogito-ddb94940-300b-4da9-99e6-1e283bf81fad">Effects</span>
text: ,
          in the Waite Houfe in Sourhamptow- -street, Bloomfbary, late in the Poffeffion of THOMAS
          MEDLYCOTT Efq; confifting ot 
tag: <span ana="#Descriptor #fine" xml:id="recogito-c3f72ccf-9d4a-42c5-a224-8508d1f5967f">fme</span>
text: 

tag: <span ana="#ma

tag: <span ana="#begin" xml:id="recogito-39330137-6545-4a31-9e93-6e8bcf59a460">To be fold by Ru ewion by Mr. COCR</span>
text: ,
By Order of the Trufires of the Hon. General BisSETer, deceor'd,
on Whitfin-Monday, the 22d inftant,
text: HE Seat of the faid General, fituate at
Twickenham in the Country of Middlefex, being Freehold, 66 Feet
in front, and 46 Feet t0 Inches deep, built in a ftron; and 
tag: <span ana="#regular #descriptor" xml:id="recogito-3dd8bda1-6da4-4210-a4c7-d437d9f6ab12">regular</span>
text:  
tag: <span ana="#manner" xml:id="recogito-2125cb1f-4cff-473e-88d6-9fee6549ee78">Manner</span>
text: ,
having five 
tag: <span ana="#large #descriptor" xml:id="recogito-6f8369e7-a8d6-4b2a-9950-7bfaa2143442">large</span>
text:  
tag: <span ana="#room" xml:id="recogito-0635871a-6b62-4bf6-9d45-65ea3a280659">Rooms</span>
text:  on a Floor with 
tag: <span ana="#dressing room" xml:id="recogito-9c8772b2-17d5-4154-bd7d-52f76d5e3f1d">Dreffing-Rooms</span>
text:  and 
tag: <span ana="#clo

tag: <span ana="#begin" xml:id="recogito-52fde243-bb9e-4a81-a8f3-81ce546fdd04">By Meffes. ELDERTONS</span>
text: ,
At Twelve o'clock,
On Thurfday the 29th Inftant,
On the Premifes,
text: HE LEASE of that 
tag: <span ana="#good #descriptor" xml:id="recogito-3509f3ba-88dc-4b1f-a9cc-3937fbae096d">good</span>
text:  and 
tag: <span ana="#desirable #descriptor" xml:id="recogito-6540ddd0-7271-4cca-bda4-8cf3dfcd3456">defirable</span>
text: 

tag: <span ana="#house" xml:id="recogito-e9643b2a-6d9e-491c-bf74-1af49b7cdd3e">Houfe</span>
text:  late in the occupation of Mr. JOSEPH
VAUX
The PACKHORSE, No. fifi, Cornhill.
The 
tag: <span ana="#premise" xml:id="recogito-8241bcb7-4ecb-42f7-a993-28adf2b34cff">premifes</span>
text:  have been built about feven years, are

tag: <span ana="#finished in the most substantial manner #descriptor" xml:id="recogito-ec79ea42-543c-44c8-a1da-e766fc6b6f52">finifhed in the moft fubftantiat manner</span>
text: , and 
tag: <span ana="#completed with great elegance" xml

tag: <span ana="#begin" xml:id="recogito-97a8527a-9d1f-4815-94b9-5ea1db61ffd7">By Mr CO</span>
text: 
On Monday next, the 16th inftant, and the following Doy.
HE 
tag: <span ana="#descriptor #household" xml:id="recogito-e4d34e4d-3608-46f9-8fcb-aa2501d5fed2">HOUSHOLD</span>
text:  
tag: <span ana="#furniture" xml:id="recogito-dd6c502e-8a93-453c-ae21-ba33403a929c">FURNITURE,</span>
text: 

tag: <span ana="#picture" xml:id="recogito-78e293e8-8ae1-4949-b5b0-97a29e2a0078">PICTURES</span>
text: , and other 
tag: <span ana="#effect" xml:id="recogito-8c7122fd-2748-4654-9537-2dc2738067d0">Effects</span>
text: , in the Waite Houfe in Sourhamptow-
-street, Bloomfbary, late in the Poffeffion of THOMAS MEDLYCOTT
Efq; confifting ot 
tag: <span ana="#fine #descriptor" xml:id="recogito-2b6e28df-fcb2-4464-b8ae-7096b59d2e8f">fme</span>
text:  
tag: <span ana="#mantua" xml:id="recogito-c62a1216-0c51-4edb-82fa-f964f6cd406f">Mantua</span>
text: , 
tag: <span ana="#worsted #descriptor" xml:id="recogito-1fc7